In [1]:
from typing import Union, List

from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import Optimizer
import tensorflow.keras.optimizers as optimizers
from tensorflow.keras.preprocessing.text import tokenizer_from_json
from tensorflow.keras.layers import InputLayer, Dense, Dropout, Embedding
from gensim.models import Word2Vec

import keras_tuner as kt
import numpy as np
import pandas as pd

2021-12-01 12:57:57.693497: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-01 12:57:57.693520: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## Load data

In [2]:
X_train = pd.read_csv('../data/prepared_train.csv')
Y_train = pd.read_csv('../data/train.csv', header=0, index_col=0)
Y_train.drop('comment_text', inplace=True, axis=1)
X_test = np.fromfile('../data/prepared_test.csv')

In [3]:
X_train.shape

(159570, 2142)

In [4]:

def choice_dict(name, hp: kt.HyperParameters, **options):
    return options[hp.Choice(name, options.keys())]

def choice_list(name, hp: kt.HyperParameters, *options):
    return options[hp.Int(name + '_index', 0, len(options) - 1)]


In [5]:
input_shape = (None, X_train.shape[1])
output_shape = (1, 5)

In [7]:

def build_model(
    w2v: Word2Vec,
    hidden_layers: int,
    hidden_layers_units: List[int],
    use_bias: bool,
    dropout: float,
    optimizer: Union[str, Optimizer],

):
    model = Sequential()

    model.add(
        Embedding(
            input_dim=len(w2v.wv),
            output_dim=w2v.vector_size,
            weights=w2v.wv.vectors
        )
    )

    for layer_index in range(hidden_layers):
        model.add(
            Dense(
                units=hidden_layers_units[layer_index],
                activation='relu',
                use_bias=use_bias
            )
        )

        if dropout != 0:
            model.add(
                Dropout(dropout)
            )

    model.add(
        Dense(
            units=output_shape[1],
            activation='sigmoid'
        )
    )

    model.compile(
        optimizer=optimizer,
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    return model

def build_model_from_hp(hp: kt.HyperParameters):
    return build_model(
        w2v=choice_dict(
            name='w2v', hp=hp,
            light=Word2Vec.load('light_word2vec.model'),
            heavy=Word2Vec.load('heavy_word2vec.model')
        ),
        hidden_layers=hp.Int('hidden_layers', 2, 10),
        hidden_layers_units=choice_list(
            'hidden_layers_units', hp,
            np.linspace(100, 10, 9, dtype=np.dtype('int64')),
            np.linspace(1000, 50, 9, dtype=np.dtype('int64')),
            np.linspace(1000, 50, 9, dtype=np.dtype('int64')),
            np.linspace(2000, 100, 9, dtype=np.dtype('int64')),
        ),
        use_bias=hp.Boolean('use_bias'),
        dropout=hp.Choice('dropout', [0., 0.3, 0.5]),
        optimizer=hp.Choice('optimizer', ['sgd', 'adam']),
    )


In [8]:
tuner = kt.RandomSearch(
    build_model_from_hp,
    objective='val_loss',
    max_trials=5
)

Traceback (most recent call last):
  File "/home/uiqkos/pyenvs/mint/lib/python3.9/site-packages/keras_tuner/engine/hypermodel.py", line 127, in build
    model = self.hypermodel.build(hp)
  File "/tmp/ipykernel_9540/1337214513.py", line 53, in build_model_from_hp
    light=Word2Vec.load('light_word2vec.model'),
  File "/home/uiqkos/pyenvs/mint/lib/python3.9/site-packages/gensim/models/word2vec.py", line 1930, in load
    model = super(Word2Vec, cls).load(*args, **kwargs)
  File "/home/uiqkos/pyenvs/mint/lib/python3.9/site-packages/gensim/utils.py", line 485, in load
    obj = unpickle(fname)
  File "/home/uiqkos/pyenvs/mint/lib/python3.9/site-packages/gensim/utils.py", line 1459, in unpickle
    with open(fname, 'rb') as f:
  File "/home/uiqkos/pyenvs/mint/lib/python3.9/site-packages/smart_open/smart_open_lib.py", line 188, in open
    fobj = _shortcut_open(
  File "/home/uiqkos/pyenvs/mint/lib/python3.9/site-packages/smart_open/smart_open_lib.py", line 361, in _shortcut_open
    retur

Invalid model 0/5
Invalid model 1/5
Invalid model 2/5
Invalid model 3/5
Invalid model 4/5
Invalid model 5/5


Traceback (most recent call last):
  File "/home/uiqkos/pyenvs/mint/lib/python3.9/site-packages/keras_tuner/engine/hypermodel.py", line 127, in build
    model = self.hypermodel.build(hp)
  File "/tmp/ipykernel_9540/1337214513.py", line 53, in build_model_from_hp
    light=Word2Vec.load('light_word2vec.model'),
  File "/home/uiqkos/pyenvs/mint/lib/python3.9/site-packages/gensim/models/word2vec.py", line 1930, in load
    model = super(Word2Vec, cls).load(*args, **kwargs)
  File "/home/uiqkos/pyenvs/mint/lib/python3.9/site-packages/gensim/utils.py", line 485, in load
    obj = unpickle(fname)
  File "/home/uiqkos/pyenvs/mint/lib/python3.9/site-packages/gensim/utils.py", line 1459, in unpickle
    with open(fname, 'rb') as f:
  File "/home/uiqkos/pyenvs/mint/lib/python3.9/site-packages/smart_open/smart_open_lib.py", line 188, in open
    fobj = _shortcut_open(
  File "/home/uiqkos/pyenvs/mint/lib/python3.9/site-packages/smart_open/smart_open_lib.py", line 361, in _shortcut_open
    retur

RuntimeError: Too many failed attempts to build model.